In [4]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA, FastICA
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import json
import pickle

In [7]:
def sum_pir(data):
    new_data = []
    for r in range(len(data)):
        new_entry = [0,0,0,0,0]
        for c in range(len(data[r])):
            new_entry[c%5] += data[r][c]
        new_data.append(new_entry)
    return new_data

with open("data_2_28.txt", "r") as file:
    lines = file.readlines()
    
data = []
y_loc = []
y_d = []
y_v = []
for i in lines:
    tmp = json.loads(i)
    data.append(tmp["values"])
    y_loc.append(tmp["y"])
    y_d.append(tmp["d"])
    y_v.append(tmp["v"])

df = pd.DataFrame(data)
df.describe()


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,0.166667,0.211310,0.241071,0.401786,0.202381,0.151786,0.217262,0.208333,0.401786,0.190476,...,0.053571,0.113095,0.125000,0.163690,0.107143,0.050595,0.113095,0.101190,0.125000,0.101190
std,0.373234,0.408846,0.428371,0.490990,0.402374,0.359348,0.412997,0.406722,0.490990,0.393262,...,0.225506,0.317182,0.331212,0.370546,0.309756,0.219497,0.317182,0.302031,0.331212,0.302031
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
X = df
ica = FastICA(n_components=5).fit(X)
print ica.components_
X = ica.transform(X)

[[ -3.36819971e-03  -1.18842608e-02   1.81320894e-02  -3.24768770e-03
    2.92571301e-03  -3.17454079e-03  -1.22367513e-02   1.89326647e-02
   -3.22599873e-03   3.50684337e-03  -2.50057268e-03  -1.32079789e-02
    1.99827000e-02  -2.58549824e-03   3.90321934e-03  -1.16850471e-03
   -1.27486559e-02   2.00732264e-02  -2.01122723e-03   4.20151267e-03
   -4.38382630e-04  -1.14334956e-02   1.82075462e-02  -1.87603065e-03
    4.86198174e-03  -1.06801665e-05  -1.02515576e-02   1.66736633e-02
   -2.24544866e-03   4.52553846e-03   3.75677695e-04  -9.37464485e-03
    1.46505222e-02  -2.32201645e-03   3.43713447e-03   8.93753563e-04
   -6.32606806e-03   1.12919740e-02  -6.59971991e-04   2.47785206e-03
    4.52941054e-05  -3.49436912e-03   9.13732947e-03   8.68724622e-04
    1.03205894e-03   5.52439755e-04  -2.11585172e-03   4.74285027e-03
    2.69185663e-03   1.14280340e-05]
 [ -2.10698923e-02   1.24008307e-02   3.24180964e-04   1.47698450e-03
    3.88423735e-03  -2.23714201e-02   1.23051610e-02 

In [9]:
param_grid = [{'n_components':range(2,11)}]
clf = GridSearchCV(GaussianMixture(),param_grid).fit(X)
clusterer = clf.best_estimator_ 
y = clusterer.predict(X)

In [10]:
print clusterer.score(X,y)
print clusterer.score_samples(X)

12.4715390638
[ 16.48352215  16.1258302   16.41138225  16.38784141  16.83375986
  17.15688588  16.38784141  16.41138225  16.93456502  16.41138225
  15.97587093  17.17478973  17.15688588  17.08026361  16.97065413
  14.52046482  16.93602946  16.44784027  16.93587192  16.48352215
  16.48352215  16.98502149  16.92981238  15.07214738  16.38561244
  16.7937769   14.93931427  13.82445465  14.91658303  16.98502149
  12.48283505  15.48958823  14.31988711  14.74008256  17.15688588
  17.0881404   16.78250659  14.05551675  16.93587192  16.7937769
  16.93587192  17.31456964  16.48352215  16.44784027  16.99158446
  17.08026361  17.31456964  11.82507318  14.29542844  16.44784027
  16.73794242  14.58084059  15.5126024   12.64470431  16.93456502
  16.44784027  16.72880899  16.51084402  14.75675038  16.66747292
  16.87854723  13.29455204  17.15688588  16.74090567  13.83525472
  16.1258302   16.97065413  16.82837658  15.29522125  15.84241339
  16.7937769   16.48352215  16.83375986  16.78250659  17.174789

In [11]:
statistics = np.around(sum_pir(ica.inverse_transform(clusterer.means_)),1)
for i in statistics:
    print i

[-0.1  0.   0.   4.   2.8]
[ 0.4  0.   4.5  0.7  0.1]
[ 0.6  5.7  2.6  1.7  1.1]
[ 6.4  2.5  1.   1.   0.1]
[ 0.1  0.2  4.4  6.   0.3]


In [7]:
labels = ["sink","hallbed","refrig","nearbedsink","ovenrefrig",\
          "across_part_room","hall","bed","across_whole_room"]

In [8]:
with open("GaussianMixture","wb") as sf1:
    pickle.dump(clusterer, sf1)

In [10]:
with open("decomposition","wb") as sf2:
    pickle.dump(ica, sf2)

In [11]:
with open("labels","wb") as sf3:
    pickle.dump(labels, sf3)